In [52]:
from scipy.optimize import brute
import numpy as np
from datetime import datetime,timedelta
import pandas as pd
import ray

In [53]:
import Learner

In [54]:
# Initial parameters
dfparam = pd.read_csv("data/param.csv")
dfparam

,DRS,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,RATIO,WCASES,WREC
0,DRS 01 - Grande São Paulo,2020-03-15,60,280000.0,0.0001,0.0001,0.0001,0.0001,80.0000,1500,0.1,0.6,0.1
1,DRS 02 - Araçatuba,2020-04-01,60,500.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0.1,0.4,0.1
2,DRS 03 - Araraquara,2020-04-01,60,2000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0.1,0.4,0.1
3,DRS 04 - Baixada Santista,2020-04-01,60,8000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0.1,0.4,0.1
4,DRS 05 - Barretos,2020-04-01,60,2000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0.1,0.4,0.1
5,DRS 06 - Bauru,2020-04-01,60,10000.0,0.0001,0.0001,4.0000,0.0001,0.0001,0,0.1,0.4,0.1
6,DRS 07 - Campinas,2020-04-01,60,20000.0,0.0001,0.0001,40.0000,0.0001,0.0001,0,0.1,0.4,0.1
7,DRS 08 - Franca,2020-04-01,60,1000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0.1,0.4,0.1
8,DRS 09 - Marília,2020-04-01,60,5000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0.1,0.4,0.1
9,DRS 10 - Piracicaba,2020-04-01,60,10000.0,0.0001,0.0001,0.0001,0.0001,1.0000,0,0.1,0.4,0.1


In [83]:
#@ray.remote
def opt(districtRegion,e0,a0,r0,d0,date,version,wcases,wrec):
    rranges = [slice(1e5,2e6,1e5),slice(-2,2,4),slice(0,1000,1000)]
    optimal = brute(fobjective,        
        ranges=rranges,
        args=(districtRegion,e0,a0,r0,d0,date,version,wcases,wrec), full_output=True, disp=True, finish=None)
    return optimal

In [100]:
from scipy.integrate import odeint

def fobjective(point,districtRegion,e0,a0,r0,d0,date,version,wcases,wrec):
    s0, deltaDate, i0 = point
    
    startNCases = 20
    ratio = 0.1
    version = 1
    cleanRecovered = False
    
    Date = datetime.strptime(date, "%Y-%m-%d")
    end_date = Date + timedelta(days=+int(deltaDate))
    dateStr=end_date.strftime("%Y-%m-%d")
    
    
    predict_range = 60
    
    
    #objective function Odeint solver
    def lossOdeint(point, data, death, s_0, e_0, a_0, i_0, r_0, d_0, startNCases, ratioRecovered, weigthCases, weigthRecov):
        size = len(data)
        beta, beta2, sigma, sigma2, sigma3, gamma, b, mu = point
        def SEAIRD(y,t):
            S = y[0]
            E = y[1]
            A = y[2]
            I = y[3]
            R = y[4]
            D = y[5]
            p=0.2
            # beta2=beta
            y0=-(beta2*A+beta*I)*S+mu*S #S
            y1=(beta2*A+beta*I)*S-sigma*E-mu*E #E
            y2=sigma*E*(1-p)-gamma*A-mu*A #A
            y3=sigma*E*p-gamma*I-sigma2*I-sigma3*I-mu*I#I
            y4=b*I+gamma*A+sigma2*I-mu*R #R
            y5=(-(y0+y1+y2+y3+y4)) #D
            return [y0,y1,y2,y3,y4,y5]

        y0=[s_0,e_0,a_0,i_0,r_0,d_0]
        tspan=np.arange(0, size, 1)
        res=odeint(SEAIRD,y0,tspan,hmax=0.01)

        l1=0
        l2=0
        l3=0
        tot=0

        for i in range(0,len(data.values)):
            if data.values[i]>startNCases:
                l1 = l1+(res[i,3] - data.values[i])**2
                l2 = l2+(res[i,5] - death.values[i])**2
                newRecovered=min(1e6,data.values[i]*ratioRecovered)
                l3 = l3+(res[i,4] - newRecovered)**2
                tot+=1
        l1=np.sqrt(l1/max(1,tot))
        l2=np.sqrt(l2/max(1,tot))
        l3=np.sqrt(l3/max(1,tot))

        #weight for cases
        u = weigthCases  #Brazil US 0.1
        w = weigthRecov
        #weight for deaths
        v = max(0,1. - u - w)

        return u*l1 + v*l2 + w*l3
    
    
    learner = Learner.Learner.remote(districtRegion, lossOdeint,end_date, 
                                     predict_range,s0, e0, a0, i0, r0, d0, 
                                     startNCases, ratio, wcases, wrec,
                                     cleanRecovered, version, savedata=True)
    
    result = learner.train.remote()
    return result.Total

In [101]:
districtRegion = "DRS 01 - Grande São Paulo"

query = dfparam.query('DRS == "{}"'.format(districtRegion)).reset_index()
parameters = np.array(query.iloc[:, 2:])[0]

startdate,predictionrange,s0,e0,a0,i0,r0,d0,START,ratio,wcases,wrec = parameters

In [102]:
version = 1

optimal = []
optimal.append(opt(districtRegion,e0,a0,r0,d0,startdate,version,wcases,wrec)) 

AttributeError: 'ray._raylet.ObjectID' object has no attribute 'Total'

2020-04-28 14:28:48,676	ERROR worker.py:1012 -- Possible unhandled error from worker: ray::Learner.train() (pid=30182, ip=192.168.0.107)
  File "pandas/_libs/index.pyx", line 111, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1614, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1622, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: datetime.datetime(2020, 3, 13, 0, 0)

During handling of the above exception, another exception occurred:

ray::Learner.train() (pid=30182, ip=192.168.0.107)
  File "/home/v/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 4842, in get_slice_bound
    slc = self.get_loc(label)
  File "/home/v/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 2648, in get_loc
    return self._engine.get_loc(self._maybe_cas